In [10]:
from datetime import datetime
import orjson
import networkx as nx
from functools import cache


class Hypergraph:

    def __init__(self, hedge_vertices: dict, hedge_data=None, vertices_data=None):
        self._vertices = set()
        self._hyperedges = set()
        for hedge, vertices in hedge_vertices.items():
            self._vertices.update(vertices)
            self._hyperedges.add(hedge)

        self._bipartite_graph = nx.Graph()

        self._bipartite_graph.add_nodes_from(
            self._vertices, bipartite='vertex')
        self._bipartite_graph.add_nodes_from(
            self._hyperedges, bipartite='hyperedge')

        if hedge_data:
            assert not (set(hedge_data.keys()) - self._hyperedges), 'hedge_data contain non-hyperedges'
            nx.set_node_attributes(self._bipartite_graph, hedge_data)

        if vertices_data:
            assert not (set(vertices_data) - self._vertices), 'vertices_data contain non-vertices'
            nx.set_node_attributes(self._bipartite_graph, vertices_data)

        for hedge, vertices in hedge_vertices.items():
            for vertex in vertices:
                self._bipartite_graph.add_edge(vertex, hedge)

    @cache
    def get_hedge_data(self, distance_attribute, hedge=None):
        if hedge:
            return self._bipartite_graph.nodes[hedge][distance_attribute]
        else:
            return {hedge: self._bipartite_graph.nodes[hedge][distance_attribute] for hedge in self._hyperedges}

    @cache
    def hyperedges(self, vertex=None):
        if vertex:
            return set(self._bipartite_graph[vertex])
        return set(self._hyperedges)
    
    @cache
    def vertices(self, hedge=None):
        if hedge:
            return set(self._bipartite_graph[hedge])
        return set(self._vertices)

    @cache
    def hyperedge_neighbors(self, hedge=None, distance_attr=None, filter_distance=lambda distance: False):
        if hedge:
            connected_hedges: set = set()
            for vertex in self.vertices(hedge):
                for next_hedge in self.hyperedges(vertex):
                    if distance_attr:
                        distance = self.get_hedge_data(distance_attr, next_hedge) - self.get_hedge_data(distance_attr, hedge)
                    else:
                        distance = None
                    if not filter_distance(distance):
                        connected_hedges.add(next_hedge)
            return connected_hedges - {hedge}
        else:
            connected_hedges: dict = {}
            for hedge in self.hyperedges():
                connected_hedges[hedge] = self.hyperedge_neighbors(hedge, distance_attr, filter_distance)
            return connected_hedges


class CommunicationNetwork(Hypergraph):

    def __init__(self, channel_dict, channel_data=None, name=None):
        super().__init__(channel_dict, channel_data)
        self.name = name

    def channels(self, participant=None):
        return self.hyperedges(participant)

    def participants(self, channel=None):
        return self.vertices(channel)

    @classmethod
    def from_json(cls, file_path: str, name=None):
        with open(file_path, 'r', encoding='utf8') as json_file:
            raw_data = orjson.loads(json_file.read())
        hedge_dict = {chan_id: {
            str(p) for p in channel['participants']} for chan_id, channel in raw_data.items()}
        hedge_data = {chan_id: {
            'start': datetime.fromisoformat(channel['start']),
            'end': datetime.fromisoformat(channel['end']),
        } for chan_id, channel in raw_data.items()}

        return cls(hedge_dict, hedge_data, name=name)


In [3]:
from tqdm.auto import tqdm

In [7]:
cn = CommunicationNetwork.from_json('../data/simulation_parameters.json')

In [14]:
len(cn.hyperedge_neighbors('432663143231462207'))

62

In [12]:
cn.hyperedges()

{'432663143231462207',
 '-8805916618625198208',
 '-1160808172053390997',
 '4543948132907691405',
 '848263050297565140',
 '5040463312835548573',
 '-2603248285721058134',
 '7977785367056842251',
 '8281987729220299423',
 '4467780632513259997',
 '2608920949631473812',
 '-6092514335773474443',
 '7592416070128159218',
 '-5016734436546047104',
 '-2336125120912237720',
 '1178965022553055416',
 '-2323512643549304415',
 '-7388536649662091954',
 '6959671176977964851',
 '1007975938377348484',
 '2155481969329205987',
 '858563195068943295',
 '2869735460876052362',
 '-8965394570826870044',
 '2909518063254341744',
 '-6901713987255631060',
 '7569193270410673304',
 '683498481085422461',
 '-3183879201174915986',
 '-5585705112935921201',
 '-5214161735460804752',
 '-6315611377671126558',
 '-6116786017422453743',
 '-6631910775593936406',
 '-9080947371726683211',
 '4086923436886094849',
 '8001281887565337689',
 '2749431751926959658',
 '-1429335723499803285',
 '1006899148705985549',
 '-2386490502060806621',
 

In [30]:
hedge_data = {'h1': {'end': datetime(2022, 1, 1)}, 'h2': {'end': datetime(2022, 1, 2)}, 'h3': {'end': datetime(2022, 1, 3)}}
cn = CommunicationNetwork({'h1': ['v1', 'v2'], 'h2': ['v2', 'v3'], 'h3': ['v3', 'v4']}, hedge_data)

In [37]:
from tqdm.auto import tqdm
from datetime import timedelta


def run_simulation_ignoring_time(cn: CommunicationNetwork):
    participants = cn.participants()
    result = {}
    with tqdm(total=len(participants), desc='Simulating ignoring time') as pbar:
        for p in participants:
            if p not in result:
                reachable_participants = bfs(cn, p)
                for _p in reachable_participants:
                    if _p not in result:
                        result[_p] = reachable_participants - {_p}
                        pbar.update()
    return result


def run_simulation_respecting_time(cn: CommunicationNetwork, cache=False):
    def filter_distance(d): return d <= timedelta(seconds=0)
    if cache:
        for channel in tqdm(cn.channels(), desc='Caching channel neighbors'):
            cn.channel_neighbors(channel, 'end', filter_distance)
    result = {}
    for p in tqdm(cn.participants(), desc='Simulating respecting time'):
        result[p] = bfs(cn, p, 'end', filter_distance)
    return result

In [39]:
run_simulation_respecting_time(cn)

Simulating respecting time:   0%|          | 0/4 [00:00<?, ?it/s]

{'v1': {'v1', 'v2', 'v3', 'v4'},
 'v3': {'v2', 'v3', 'v4'},
 'v2': {'v1', 'v2', 'v3', 'v4'},
 'v4': {'v3', 'v4'}}

In [4]:
from collections import deque
from datetime import timedelta

In [16]:
hedge_data = {'h1': {'t': 1}, 'h2': {'t': 2}, 'h3': {'t': 3}}
cn = CommunicationNetwork({'h1': ['v1', 'v2'], 'h2': ['v2', 'v3'], 'h3': ['v3', 'v4']}, hedge_data)

In [29]:
r = {}
for v in cn.vertices():
    r[v] = bfs(cn, v, 't', lambda d: d<=0) - {v}
r

{'v1': {'v2', 'v3', 'v4'},
 'v3': {'v2', 'v4'},
 'v2': {'v1', 'v3', 'v4'},
 'v4': {'v3'}}

In [22]:
bfs(cn, 'v1')

{'v1', 'v2', 'v3', 'v4'}

In [20]:
from collections import deque
def bfs(hypergraph: Hypergraph, source_vertex, distance_attr=None, filter_distance=lambda distance: False):
    d = deque()
    for source_hedge in hypergraph.hyperedges(source_vertex):
        d.append(source_hedge)

    visited_hedges = set()
    seen_vertices = set()
    while d:
        hedge = d.pop()
        seen_vertices |= hypergraph.vertices(hedge)
        for next_hedge in hypergraph.hyperedge_neighbors(hedge, distance_attr, filter_distance):
            if next_hedge not in visited_hedges:
                visited_hedges.add(next_hedge)
                d.append(next_hedge)
    
    return seen_vertices

In [11]:
h = Hypergraph({'h1': ['v1', 'v2'], 'h2': ['v2', 'v3'], 'h3': ['v3', 'v4']}, {'h1': {'t': 1}, 'h2': {'t': 2}, 'h3': {'t': 3}})

In [12]:
h.hyperedge_neighbors()

{'h2': {'h1', 'h3'}, 'h1': {'h2'}, 'h3': {'h2'}}

In [51]:
bfs(h, 'v2')

{'v1', 'v2', 'v3', 'v4'}

In [6]:
all_participants = list(cn.vertices())
selected_participants = all_participants[:100]

In [31]:
ti_upper_bound = {}
with tqdm(total=len(all_participants)) as pbar:
    for p in all_participants:
        if p not in ti_upper_bound:
            conncect_component = bfs(cn, p)
            for _p in conncect_component:
                if _p not in ti_upper_bound:
                    ti_upper_bound[_p] = conncect_component - {_p}
                    pbar.update()

  0%|          | 0/37103 [00:00<?, ?it/s]

In [44]:
store_result(ti, 'conncect_component.json')

TypeError: Type is not JSON serializable: set

In [12]:
tr = {}
for p in tqdm(selected_participants):
    tr[p] = bfs(cn, p, 'end')    

  0%|          | 0/100 [00:00<?, ?it/s]

KeyError: <function bfs.<locals>.<lambda> at 0x150c23d90>

In [41]:
import orjson
from pathlib import Path

def decode_result(result: dict) -> bytes:
    def _default(obj):
        if isinstance(obj, set):
            return {o: None for o in sorted(obj)}
        raise TypeError
    return orjson.dumps(result, default=_default, option=orjson.OPT_SORT_KEYS)


def store_result(result: dict, file_name):
    file_path = Path('..')/'results'/file_name
    with open(file_path, 'wb') as f:
        f.write(decode_result(result))

In [93]:
store_result(r, 'horizon.json')